In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data/processed_data.csv')

In [4]:
df.head(10)

,No.,Time,Source,Destination,Protocol,Length,Info,label
0,1,0.000000,2620:1ec:50::12,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,TLSv1.2,113,Application Data,0
1,2,0.009874,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,2620:1ec:50::12,TLSv1.2,109,Application Data,0
2,3,0.010204,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,2620:1ec:50::12,TLSv1.2,109,Application Data,0
3,4,0.056850,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,2404:6800:4003:c03::bc,TCP,75,61743 > 5228 [ACK] Seq=1 Ack=1 Win=256 Len=1,0
4,5,0.204813,2620:1ec:50::12,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,TCP,74,443 > 61742 [ACK] Seq=40 Ack=71 Win=16386 Len=0,0
5,6,0.204813,2404:6800:4003:c03::bc,2401:4900:1a54:1802:6144:3bc9:3fe2:2fa1,TCP,86,5228 > 61743 [ACK] Seq=1 Ack=2 Win=1045 Len=...,0
6,7,8.601737,fe80::745a:18ff:fe00:395c,ff02::16,ICMPv6,90,Multicast Listener Report Message v2,1
7,8,8.602882,192.168.43.1,224.0.0.22,IGMPv3,54,Membership Report / Join group 224.0.0.251 for...,1
8,9,8.805984,192.168.43.1,224.0.0.22,IGMPv3,54,Membership Report / Join group 224.0.0.251 for...,1
9,10,9.013957,fe80::745a:18ff:fe00:395c,ff02::fb,MDNS,177,"Standard query 0x0000 ANY Android.local, ""QU"" ...",1


In [5]:
X = df.drop(['Time', 'label'], axis=1)  # Remove timestamp
y = df['label']  # Labels: 0 (Normal), 1 (Intrusion)

In [12]:
# Example: encode each column in X_train
encoder = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = encoder.fit_transform(X[col])

In [20]:
X.head(10)

,No.,Source,Destination,Protocol,Length,Info
0,1,28,21,9,113,1632
1,2,19,46,9,109,1632
2,3,19,46,9,109,1632
3,4,19,23,8,75,675
4,5,28,21,8,74,41
5,6,20,21,8,86,617
6,7,44,66,4,90,1721
7,8,10,16,5,54,1720
8,9,10,16,5,54,1720
9,10,44,69,6,177,1741


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
rf = RandomForestClassifier(n_estimators=100)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [16]:
ensemble = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')
ensemble.fit(X_train, y_train)

C:\Users\ishit\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:57:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')

In [17]:
# Save the trained model
with open('models/intrusion_model.pkl', 'wb') as f:
    pickle.dump(ensemble, f)

In [21]:
ensemble.predict([[9,10,16,5,54,1720]])

C:\Users\ishit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)